# CodeArena (C4) Question Answer bot

### Objective
- This notebook has the PoC work for a Question Answer bot using C4's knowledge bases.
- The objective of the PoC is to prototype an LLM implementation that can accurately answer questions to their expectation and at the very least perform better than their current bot from [Mava](https://www.mava.app/)

### Observations from the usage of Mava
- The platform offers Discord support management with ticketing and AI help bot features
- For the AI help bot, the user is able to specify links to multiple knowledge sources that can be used for answering questions.
- Based on C4's testing of the Mava bot in the private channel, the following stats were observed:-
    - Total questions asked: 29
    - Total questions mis-answered based on emoji reactions: 13
    - Accuracy - ~55%

### Knowledge Bases
Based on conversations with their team, the following knowledge bases were identified to be relevant and are the same ones that Mava is using:-
- [Main Website](https://code4rena.com/)
- [Docs](https://docs.code4rena.com/) 


### High-level Approach
- Crawl and scrape C4’s website and docs using Scrapy lib
- Convert the html content to markdown format so that the model can better understand the context
- Use LangChain lib to do the following:-
    - Split the markdown header-separated sections into semantic chunks
    - Embed and store the semantic chunks in an in-memory vector db
    - Use the retrieval augmented functionality to answer the question

In [59]:
# Install all the third-party packages

!pip install 'langchain[llms]'
!pip install Scrapy
!pip install html2text
!pip install lxml
!pip install python-dotenv
!pip install "unstructured[all-docs]"
!pip install tiktoken
!pip install faiss-cpu 
!pip install GitPython
!pip install notebook
!pip install chromadb
!pip install pandas

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.8 requires tqdm>=4.65.0, but you have tqdm 4.64.1 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install 

In [27]:
# General setup - you can specify OPENAI_API_KEY in .env file

import logging
from dotenv import load_dotenv
from IPython.display import display, Markdown, Latex

logging.getLogger().setLevel(logging.INFO)
load_dotenv()

True

In [28]:
import getpass
import os

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or getpass.getpass('Enter your OpenAI API key: ')

assert OPENAI_API_KEY, "Please set OPENAI_API_KEY in your environment variables"

In [29]:
# Paths to the data

C4_WEBSITE_STORAGE_DIR = "knowledge_base/c4/website"
C4_DOCS_STORAGE_DIR = "knowledge_base/c4/docs"
C4_GH_DOCS_STORAGE_DIR = "knowledge_base/c4/gh_docs"

### Crawling and Scraping using Scrapy

In [30]:
import os
import scrapy
import html2text
import lxml.html
import json
from urllib.parse import urlparse

class GenericSpider(scrapy.Spider):
    name = 'generic'

    def __init__(self, domain='', storage_dir='.', *args, **kwargs):
        super(GenericSpider, self).__init__(*args, **kwargs)
        self.allowed_domains = [domain]
        self.start_urls = [f'http://{domain}/']
        self.storage_dir = storage_dir
    
    def parse(self, response):
        # Remove unwanted elements using lxml
        tree = lxml.html.fromstring(response.text)
        
        # Remove non-text related tags
        for unwanted in tree.xpath('//script|//img|//video|//audio|//iframe|//object|//embed|//canvas|//svg|//link|//source|//track|//map|//area'):
            unwanted.drop_tree()

        cleaned_html = lxml.html.tostring(tree).decode('utf-8')

        # Convert HTML to Markdown
        converter = html2text.HTML2Text()
        markdown_text = converter.handle(cleaned_html)

        # Save to a markdown file in the specified directory
        if not os.path.exists(self.storage_dir):
            os.makedirs(self.storage_dir)

        url = response.url
        page_name = response.url.split("/")[-1] if response.url.split("/")[-1] else "index"

        filename = os.path.join(self.storage_dir, f'{page_name}.json')

        with open(filename, 'w') as f:
            # Store the URL and markdown text in JSON format
            json.dump({'url': url, 'md_content': markdown_text}, f)

        # Recursively follow relative links to other pages on the same domain
        for href in response.css('a::attr(href)').getall():
            url = response.urljoin(href)
            if urlparse(url).netloc in self.allowed_domains:
                yield scrapy.Request(url, self.parse)


NOTE: Data has already been scraped and saved locally as JSON files in the 'knowledge_base/c4' directory. To re-run the scraping, uncomment the code in the cell below.

On re-running the crawler, if you get 'ReactorNotRestartable' error, the notebook kernel would need to be restarted.

In [11]:
# from scrapy.crawler import CrawlerRunner
# from scrapy.utils.project import get_project_settings
# from twisted.internet import reactor

# settings = get_project_settings()

# runner = CrawlerRunner(settings)
# runner.crawl(GenericSpider, domain="code4rena.com", storage_dir=C4_WEBSITE_STORAGE_DIR)
# runner.crawl(GenericSpider, domain="docs.code4rena.com", storage_dir=C4_DOCS_STORAGE_DIR)
# d = runner.join()
# d.addBoth(lambda _: reactor.stop())
# reactor.run()

#### Get docs from Github Repo

In [ ]:
# from git import Repo

# repo = Repo.clone_from(
#     "https://github.com/code-423n4/docs", to_path=C4_GH_DOCS_STORAGE_DIR
# )

### Retrieval Augmented Generation using LangChain

#### Load locally saved scraped data

In [31]:
import json
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

def load_json_files(dir):
    loader = DirectoryLoader(dir, loader_cls=TextLoader)
    documents = loader.load()
    for d in documents:
        page_content_dict = json.loads(d.page_content)
        d.page_content = page_content_dict['md_content']
        d.metadata['url'] = page_content_dict['url']
    return documents

c4_website_data_list = load_json_files(C4_WEBSITE_STORAGE_DIR)
c4_docs_data_list = load_json_files(C4_DOCS_STORAGE_DIR)

In [32]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

loader = DirectoryLoader(C4_GH_DOCS_STORAGE_DIR, loader_cls=TextLoader)
c4_gh_docs_data_list = loader.load()


#### Split the markdown content into semantic chunks

In [33]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=2000, chunk_overlap=200
)


website_chunks =  md_splitter.split_documents(c4_website_data_list)
docs_chunks =  md_splitter.split_documents(c4_docs_data_list)
gh_docs_chunks = md_splitter.split_documents(c4_gh_docs_data_list)

print(len(website_chunks))
print(len(docs_chunks))
print(len(gh_docs_chunks))

89
97
72


#### Embed the semantic chunks and store in an in-memory vector db

In [34]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# NOTE: At times, OpenAI Embedding service can fail intermittently and return errorneous values such as [NaN], more info: https://github.com/langchain-ai/langchain/pull/7070

embeddings = OpenAIEmbeddings()
vectorstore = Chroma("langchain_store", embeddings)

vectorstore.add_documents(website_chunks)
# vectorstore.add_documents(docs_chunks)
vectorstore.add_documents(gh_docs_chunks)


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: OpenAI API returned an empty embedding.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: OpenAI API returned an empty embedding.


['9f122ac2-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122bf8-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122c52-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122c8e-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122cca-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122cfc-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122d38-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122d6a-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122d9c-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122dce-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122e00-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122e32-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122e64-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122e96-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122ebe-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122ef0-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122f22-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122f54-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122f86-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122fb8-50d5-11ee-8d9d-367dda1ae1c5',
 '9f122fea-50d5-11ee-8d9d-367dda1ae1c5',
 '9f12301c-50d5-11ee-8d9d-367dda1ae1c5',
 '9f12304e-50d5-11ee-8d9d-367dda1ae1c5',
 '9f123080-50d5-11ee-8d9d-367dda1ae1c5',
 '9f1230b2-50d5-

#### Retrieval QA chain

In [53]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4", temperature=0), chain_type="stuff", retriever=vectorstore.as_retriever(), return_source_documents=True)


def call_llm(question):
    result = qa({"query": question})
    return result
 

def ask(question):
    result = call_llm(question)
    display(Markdown(f"### Question"))
    display(Markdown(question))

    display(Markdown(f"### Answer"))
    display(Markdown(result["result"]))

    display(Markdown(f"### Sources"))
    sources = [r.metadata['url'] if 'url' in r.metadata else r.metadata['source'] for r in result["source_documents"] ]
    print(", ".join(sources))

In [23]:
# Questions that were answered incorrectly by the Mava bot as per emoji reaction in the test channel
MAVA_MISANSWERED_QUES = [
    "what's a scout?",
    "Am I allowed to use AI in an audit?",
    "Can I change my Code4rena username?",
    "How do I book a solo audit?",
    "Do I need to be certified to participate in an audit?",
    "How do bot races work?",
    "Can I change my Code4rena profile name?",
    "What are scout awards?",
    "What are analysis reports?",
    "what is an analysis finding?",
    "My name wasn't in the award announcements. When can I check on my results?",
    "How long does the certification process take?",
    "How can I access findings.csv?"
]

In [ ]:
# for q in MAVA_MISANSWERED_QUES:
#     ask(q)

#### AutoEvaluator
Using LangChain's [AutoEvaluator technique](https://autoevaluator.langchain.com/) to evaluate the bot's performance on the dataset of C4 questions correctly answered by Mava as per team feedback


In [96]:
import yaml

# load yaml file
with open('knowledge_base/c4/c4_test_qa.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)

mava_questions = [d['question'] for d in yaml_data]


In [97]:
from langchain.prompts import PromptTemplate

template = """ 
    You are a grader trying to determine if a set of retrieved documents will help a student answer a question. \n

    Here is the question: \n
    {query}

    Here are the documents retrieved to answer question: \n
    {result}
    
    Here is the correct answer to the question: \n 
    {answer}
   
    Criteria: 
      relevance: Do all of the documents contain information that will help the student arrive that the correct answer to the question?"

    Your response should be as follows:

    GRADE: (Correct or Incorrect, depending if all of the documents retrieved meet the criterion)
    (line break)
    JUSTIFICATION: (Write out in a step by step manner your reasoning about the criterion to be sure that your conclusion is correct. Use three sentences maximum. Keep the answer as concise as possible.)
    """

GRADE_DOCS_PROMPT = PromptTemplate(input_variables=['result', 'answer', 'query'], template=template)

template = """You are a teacher grading a quiz. 
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either Correct or Incorrect.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: Correct or Incorrect here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. If the student answers that there is no specific information provided in the context, then the answer is Incorrect. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:

Your response should be as follows:

GRADE: (Correct or Incorrect)
(line break)
JUSTIFICATION: (Without mentioning the student/teacher framing of this prompt, explain why the STUDENT ANSWER is Correct or Incorrect. Use one or two sentences maximum. Keep the answer as concise as possible.)
"""

GRADE_ANSWER_PROMPT = PromptTemplate(input_variables=["query", "result", "answer"], template=template)

In [98]:
from langchain.evaluation.qa import QAEvalChain

def grade_model_answer(predicted_dataset, predictions):

    # Create an evaluation chain
    eval_chain = QAEvalChain.from_llm(
        llm=ChatOpenAI(model_name="gpt-4", temperature=0),
        prompt=GRADE_ANSWER_PROMPT
    )

    # Evaluate the predictions and ground truth using the evaluation chain
    graded_outputs = eval_chain.evaluate(
        predicted_dataset,
        predictions,
        question_key="question",
        prediction_key="result"
    )

    return graded_outputs


def grade_model_retrieval(gt_dataset, predictions):
    # Create an evaluation chain
    eval_chain = QAEvalChain.from_llm(
        llm=ChatOpenAI(model_name="gpt-4", temperature=0),
        prompt=GRADE_DOCS_PROMPT
    )

    # Evaluate the predictions and ground truth using the evaluation chain
    graded_outputs = eval_chain.evaluate(
        gt_dataset,
        predictions,
        question_key="question",
        prediction_key="result"
    )
    return graded_outputs

In [99]:
bot_answers = []
source_docs = []
for d in yaml_data:
    result = call_llm(d['question'])
    bot_answers.append(result['result'])
    source_docs.append(result['source_documents'])


In [100]:
predictions = [{'result': a} for a in bot_answers]

answer_grades = grade_model_retrieval(yaml_data, predictions)

In [102]:
retrieved_docs = []
for i, d in enumerate(yaml_data):
    retrieved_doc_text = ""
    for j, doc in enumerate(source_docs[i]):
        retrieved_doc_text += "Doc %s: " % str(j + 1) + doc.page_content + " "
    retrieved = {"question": d["question"], "answer": d["answer"], "result": retrieved_doc_text}
    retrieved_docs.append(retrieved)

retrieval_grades = grade_model_retrieval(yaml_data, predictions)

In [109]:
import pandas as pd

df = pd.DataFrame({
    "question": [d['question'] for d in yaml_data],
    "Mava correct answer (True value)": [d['answer'] for d in yaml_data],
    "Bot answers": [p['result'] for p in predictions],
    "Answer similarity score": ['Incorrect' if 'Incorrect' in g['results'] else 'Correct' for g in answer_grades],
    "Retrieval relevancy score": ['Incorrect' if 'Incorrect' in g['results'] else 'Correct' for g in retrieval_grades]
})
df

,question,Mava correct answer (True value),Bot answers,Answer similarity score,Retrieval relevancy score
0,"Hi, how can I get backstage access?","To get backstage access, you need to become a ...","I'm sorry, but the provided context does not c...",Incorrect,Incorrect
1,how long does it take until findings are relea...,"Based on the context provided, the findings fr...","The audit report, which includes the findings,...",Correct,Correct
2,When can I talk about findings?,You can talk about your findings after the con...,The context does not provide information on wh...,Incorrect,Incorrect
3,How do I change my wallet address?,"To change your wallet address, follow these st...","Unfortunately, due to some restrictions in Mor...",Incorrect,Incorrect
4,What are scouts?,"In the context of Code4rena, Scouts are indivi...",The text provided does not provide any informa...,Incorrect,Incorrect
5,How long does the contest process usually take?,"Based on the provided context, the contest pro...","The contest process, from the closing of audit...",Correct,Correct
6,how does certification work?,The certification process at Code4rena works i...,Certification for wardens involves a process w...,Correct,Correct
7,Can I use bots to analyze code?,"Yes, you can use bots to analyze code. In fact...","Yes, you can use bots to analyze code. In fact...",Correct,Correct
8,Can I use bots to analyze code?,"Yes, you can use bots to analyze code. In fact...","Yes, you can use bots to analyze code. In fact...",Correct,Correct
9,Can I use chatgpt?,"Yes, you can use bots to analyze code. This is...",The provided context does not contain informat...,Incorrect,Incorrect


In [110]:
print(f"Bot Accuracy: {df['Answer similarity score'].value_counts()['Correct'] / len(df['Answer similarity score'])}")

Bot Accuracy: 0.5454545454545454


55